# Train OCR text Detector quick example

For train datasets please download last version of ocr datasets [https://nomeroff.net.ua/datasets/](https://nomeroff.net.ua/datasets/). Unpack archive and rename to **./datasets/ocr** .
For examle
```bash
cd ./datasets/ocr
wget https://nomeroff.net.ua/datasets/autoriaNumberplateOcrBy-2020-10-09.zip
unzip autoriaNumberplateOcrBy-2020-10-09.zip
mv autoriaNumberplateOcrBy-2020-10-09 by
```
or use your own dataset.

In [1]:
# Specify device
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0" 
os.environ["TF_FORCE_GPU_ALLOW_GROWTH"]="true"

In [2]:
# from tensorflow.compat.v1 import ConfigProto
# from tensorflow.compat.v1 import InteractiveSession

# config = ConfigProto()
# config.gpu_options.allow_growth = True
# session = InteractiveSession(config=config)

In [3]:
import os
import sys
import warnings
warnings.filterwarnings('ignore')

# change this property
NOMEROFF_NET_DIR = os.path.abspath('../')

DATASET_NAME = "by"
VERSION = "2021_08_28_tensorflow_v2.3"
PATH_TO_DATASET = os.path.join(NOMEROFF_NET_DIR, "datasets/ocr/", DATASET_NAME)
RESULT_MODEL_PATH = os.path.join(NOMEROFF_NET_DIR, "models/", 'anpr_ocr_{}_{}.h5'.format(DATASET_NAME, VERSION))

sys.path.append(NOMEROFF_NET_DIR)

from NomeroffNet.Base import OCR

In [4]:
class by(OCR):
    def __init__(self):
        OCR.__init__(self)
        # only for usage model
        # in train generate automaticly
        self.letters = ["0", "1", "2", "3", "4", "5", "6", "7", "8", "9", "A", "B", "C", "E", "H", "I", "K", "M", "O", "P", "T", "X"]
        
        self.EPOCHS = 2

In [5]:
ocrTextDetector = by()

In [9]:
#ocrTextDetector.load('/var/www/nomeroff-net23/models/anpr_ocr_by_2021_07_29_tensorflow_v2.3_1.h5', verbose=False)
ocrTextDetector.load('/var/www/nomeroff-net23/models/1/anpr_ocr_by_2021_08_29_tensorflow_v2.3.h5', verbose=False)

In [7]:
#ocrTextDetector = by()
model = ocrTextDetector.prepare(PATH_TO_DATASET, use_aug=False)

GET ALPHABET
Max plate length in "val": 7
Max plate length in "train": 7
Max plate length in "test": 7
Letters train  {'M', '8', 'C', 'X', '1', '4', '0', 'H', 'I', '5', 'K', '7', 'B', 'P', 'A', 'T', '3', '6', 'O', '2', 'E', '9'}
Letters val  {'M', '8', 'C', 'X', '1', '4', '0', 'H', 'I', '5', 'K', '7', 'B', 'P', 'A', 'T', '3', '6', 'O', '2', 'E', '9'}
Letters test  {'M', '8', 'C', 'X', '1', '4', '0', 'H', 'I', '5', 'K', '7', 'B', 'A', 'P', 'T', '3', '6', 'O', '2', 'E', '9'}
Max plate length in train, test and val do match
Letters in train, val and test do match
Letters: 0 1 2 3 4 5 6 7 8 9 A B C E H I K M O P T X

EXPLAIN DATA TRANSFORMATIONS
Text generator output (data which will be fed into the neutral network):
1) the_input (image)
2) the_labels (plate number): 8127EM7 is encoded as [8, 1, 2, 7, 13, 17, 7]
3) input_length (width of image that is fed to the loss function): 30 == 128 / 4 - 2
4) label_length (length of plate number): 7
START BUILD DATA
DATA PREPARED


In [8]:
# A = {'M', 'P', 'X', 'I', 'T', 'C', '5', '3', '6', '0', '7', '2', '8', 'С', 'E', 'B', '1', 'H', 'O', 'K', 'A', '4', '9'}
# B = {'P', 'M', 'X', 'I', 'T', 'C', '5', '3', '6', '0', '7', '2', '8', 'E', 'B', '1', 'H', 'O', 'K', 'A', '4', '9'}
# A-B

In [9]:
model = ocrTextDetector.train()


START TRAINING
Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
the_input_by (InputLayer)       [(None, 128, 64, 1)] 0                                            
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 128, 64, 16)  160         the_input_by[0][0]               
__________________________________________________________________________________________________
max1 (MaxPooling2D)             (None, 64, 32, 16)   0           conv1[0][0]                      
__________________________________________________________________________________________________
conv2 (Conv2D)                  (None, 64, 32, 16)   2320        max1[0][0]                       
____________________________________________________________________________

In [10]:
ocrTextDetector.test(verbose=True)


RUN TEST
[INFO] net_inp KerasTensor(type_spec=TensorSpec(shape=(None, 128, 64, 1), dtype=tf.float32, name='the_input_by'), name='the_input_by', description="created by layer 'the_input_by'")
[INFO] net_out KerasTensor(type_spec=TensorSpec(shape=(None, 32, 23), dtype=tf.float32, name=None), name='softmax_by/Softmax:0', description="created by layer 'softmax_by'")

Predicted: 		 9349CC7C7
True: 			 9349CC7

Predicted: 		 7805KC5
True: 			 7885KC5

Predicted: 		 6927CM7
True: 			 5927CM7

Predicted: 		 15410CE7
True: 			 1533CE7

Predicted: 		 A14875
True: 			 AT14875

Predicted: 		 8721KX7
True: 			 6721KX7

Predicted: 		 93000EM7
True: 			 3000EM7

Predicted: 		 1828MC5
True: 			 1828MC7

Predicted: 		 1717B7
True: 			 1717EH7

Predicted: 		 1334PI7
True: 			 1331PI7

Predicted: 		 76591HA7
True: 			 7659HA7

Predicted: 		 8911MI7
True: 			 8811MI7
Test processing time: 1.3975989818572998 seconds
acc: 0.9900414937759336


In [11]:
ocrTextDetector.save(RESULT_MODEL_PATH, verbose=True)

SAVED TO /var/www/nomeroff-net23/models/anpr_ocr_by_2021_08_27_tensorflow_v2.3.h5


In [ ]:
# Train with aug

In [ ]:
for i in range(11,14):
    ocrTextDetector = by()
    ocrTextDetector.EPOCHS = 1

    model = ocrTextDetector.prepare(PATH_TO_DATASET, use_aug=True)

    model = ocrTextDetector.train(load_last_weights=True, is_random=i)    
    ocrTextDetector.test(verbose=True)

    ocrTextDetector.save(os.path.join(NOMEROFF_NET_DIR, "models/", 'anpr_ocr_{}_{}_{}.h5'.format(DATASET_NAME, VERSION, i)), verbose=True)    
    

GET ALPHABET
Max plate length in "val": 7
Max plate length in "train": 7
Max plate length in "test": 7
Letters train  {'O', '9', '5', '0', 'T', '1', 'A', 'P', 'H', 'B', '8', '7', 'X', 'C', 'K', 'E', '3', 'I', '2', 'M', '4', '6'}
Letters val  {'9', 'O', '5', '0', 'T', '1', 'A', 'P', 'B', 'H', '8', '7', 'X', 'C', 'K', 'E', '3', 'I', '2', 'M', '4', '6'}
Letters test  {'O', '9', '5', '0', 'T', '1', 'A', 'P', 'B', 'H', '8', '7', 'X', 'C', 'K', 'E', '3', 'I', '2', 'M', '4', '6'}
Max plate length in train, test and val do match
Letters in train, val and test do match
Letters: 0 1 2 3 4 5 6 7 8 9 A B C E H I K M O P T X

EXPLAIN DATA TRANSFORMATIONS
Text generator output (data which will be fed into the neutral network):
1) the_input (image)
2) the_labels (plate number): 8127EM7 is encoded as [8, 1, 2, 7, 13, 17, 7]
3) input_length (width of image that is fed to the loss function): 30 == 128 / 4 - 2
4) label_length (length of plate number): 7
START BUILD DATA
DATA PREPARED

START TRAINING
Mode

161177/161177 [==============================] - 2543s 16ms/step - loss: 0.0020 - val_loss: 0.1136

RUN TEST
[INFO] net_inp KerasTensor(type_spec=TensorSpec(shape=(None, 128, 64, 1), dtype=tf.float32, name='the_input_by'), name='the_input_by', description="created by layer 'the_input_by'")
[INFO] net_out KerasTensor(type_spec=TensorSpec(shape=(None, 32, 23), dtype=tf.float32, name=None), name='softmax_by/Softmax:0', description="created by layer 'softmax_by'")

Predicted: 		 69217CM7
True: 			 5927CM7

Predicted: 		 1580CE7
True: 			 1533CE7

Predicted: 		 1023PI7
True: 			 1021PI7

Predicted: 		 9110KM4
True: 			 9119KM4

Predicted: 		 55296CK7
True: 			 5296CK7

Predicted: 		 5731CT7
True: 			 5731CI7

Predicted: 		 7162CH7
True: 			 7167CH7

Predicted: 		 8776HA7
True: 			 0776HA7

Predicted: 		 4713KO7
True: 			 4213KO7

Predicted: 		 41828MC7
True: 			 1828MC7

Predicted: 		 1637OA7
True: 			 3637OA7

Predicted: 		 9877IA6
True: 			 9877IA2

Predicted: 		 AI94144
True: 			 AI84144

In [26]:
#ocrTextDetector.load('/var/www/nomeroff-net23/models/anpr_ocr_by_2021_08_27_tensorflow_v2.3_0.h5', verbose=False)
ocrTextDetector.test(verbose=True)


RUN TEST
[INFO] net_inp KerasTensor(type_spec=TensorSpec(shape=(None, 128, 64, 1), dtype=tf.float32, name='the_input_by'), name='the_input_by', description="created by layer 'the_input_by'")
[INFO] net_out KerasTensor(type_spec=TensorSpec(shape=(None, 32, 23), dtype=tf.float32, name=None), name='softmax_by/Softmax:0', description="created by layer 'softmax_by'")

Predicted: 		 19349CC7
True: 			 9349CC7

Predicted: 		 6927CM7
True: 			 5927CM7

Predicted: 		 133CET7
True: 			 1533CE7

Predicted: 		 8721KX7
True: 			 6721KX7

Predicted: 		 9110KI4
True: 			 9119KM4

Predicted: 		 8113EH5
True: 			 9113EH5

Predicted: 		 43213KO7
True: 			 4213KO7

Predicted: 		 3637MOA7
True: 			 3637OA7

Predicted: 		 8877IAX7
True: 			 9877IA2

Predicted: 		 AI8414I6
True: 			 AI84144

Predicted: 		 454605
True: 			 AM54605

Predicted: 		 1717H7
True: 			 1717EH7

Predicted: 		 0546IC7
True: 			 0540IC7

Predicted: 		 8318A7
True: 			 8318TA7

Predicted: 		 2931HK7
True: 			 3931HK7
Test processing t

In [ ]:
ocrTextDetector.save(RESULT_MODEL_PATH, verbose=True)

In [ ]:
ocrTextDetector.load(RESULT_MODEL_PATH, verbose=False)

In [ ]:
ocrTextDetector.load('/home/www/nomeroff-net/models/anpr_ocr_ru_2020_10_07_tensorflow_v2.3.h5', verbose=True)

In [ ]:
from tensorflow.python.compiler.tensorrt import trt_convert as trt
converter = trt.TrtGraphConverterV2(input_saved_model_dir=RESULT_MODEL_PATH)
converter.convert()
converter.save(output_saved_model_dir)

In [ ]:
converter.convert()

In [ ]:
from NomeroffNet.Base import convert_keras_to_freeze_pb

In [ ]:
RESULT_MODEL_PATH1 = '/var/www/nomeroff-net/models/anpr_ocr_ru_2020_09_09_tensorflow_v2'
RESULT_MODEL_PATH_TRT = '/var/www/nomeroff-net/models/anpr_ocr_ru_2020_09_09_tensorflow_v2_trt'

In [ ]:
#ocrTextDetector.save(RESULT_MODEL_PATH1, verbose=True)

In [ ]:
from tensorflow.python.compiler.tensorrt import trt_convert as trt
converter = trt.TrtGraphConverterV2(input_saved_model_dir=RESULT_MODEL_PATH1)
converter.convert()
converter.save(RESULT_MODEL_PATH_TRT)